In [88]:
import pandas as pd

data=pd.read_csv('C:/Users/DEEPADHARSHINI/OneDrive/Desktop/Tourism/MergedTourismData.csv')

data.head()

,TransactionId,UserId,ContinentId,Continent,RegionId,Region,CountryId,Country,CityId,CityName,AttractionId,Attraction,AttractionAddress,AttractionTypeId,AttractionType,VisitYear,VisitMonth,VisitMode,VisitModeName,Rating
0,3,70456,5,Europe,21,Western Europe,163,United Kingdom,4341,Guildford,640,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",63,Nature & Wildlife Areas,2022,10,2,Couples,5
1,9,79069,2,America,9,South America,54,Brazil,774,Brazil,640,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",63,Nature & Wildlife Areas,2022,10,3,Family,5
2,18,43471,2,America,8,Northern America,51,United States,1806,San Francisco,640,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",63,Nature & Wildlife Areas,2022,10,4,Friends,5
3,29,62907,2,America,8,Northern America,51,United States,50,Alexandria,640,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",63,Nature & Wildlife Areas,2022,9,2,Couples,5
4,39,38726,4,Australia & Oceania,16,Oceania,115,New Zealand,5506,Christchurch,640,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",63,Nature & Wildlife Areas,2022,9,4,Friends,3


In [ ]:
data.info()

In [ ]:
missing_data=data.isnull().sum()
print(missing_data)

In [ ]:
data.duplicated().sum()

In [ ]:
data.shape   #rows,columns

In [ ]:
data.describe()

In [89]:
import numpy as np
from category_encoders import TargetEncoder
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,mean_absolute_error, r2_score
from sklearn.model_selection import RandomizedSearchCV


In [90]:
# SELECTING RELEVANT FEATURES FOR MODEL TRAINING, INCLUDING VISIT DETAILS, ATTRACTION INFORMATION, AND LOCATION DATA  
selected_features = [
    "VisitYear",        
    "VisitMonth",       
    "VisitModeName",         
    "Attraction",      
    "AttractionType",  
    "CountryId",        
    "RegionId"          
]

In [91]:
en_features=['VisitModeName','AttractionType']

df_selected=data[selected_features + ['Rating']]

ohe=OneHotEncoder(sparse_output=False,handle_unknown='ignore')
en_values=ohe.fit_transform(df_selected[en_features])
encoded_df=pd.DataFrame(en_values,columns=ohe.get_feature_names_out(en_features))

te=TargetEncoder()
df_selected['Attraction']=te.fit_transform(df_selected['Attraction'],df_selected['Rating'])

df_selected=df_selected.drop(columns=en_features)
df_selected=pd.concat([df_selected,encoded_df],axis=1)


print(df_selected)

C:\Users\DEEPADHARSHINI\AppData\Local\Temp\ipykernel_22480\4079427820.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['Attraction']=te.fit_transform(df_selected['Attraction'],df_selected['Rating'])


       VisitYear  VisitMonth  Attraction  CountryId  RegionId  Rating  \
0           2022          10    4.267030        163        21       5   
1           2022          10    4.267030         54         9       5   
2           2022          10    4.267030         51         8       5   
3           2022           9    4.267030         51         8       5   
4           2022           9    4.267030        115        16       3   
...          ...         ...         ...        ...       ...     ...   
52917       2013           5    4.646601        109        15       3   
52918       2014           3    4.646601        109        15       5   
52919       2013          11    4.646601         93        13       4   
52920       2013           9    4.646601        115        16       5   
52921       2014           1    4.646601        109        15       3   

       VisitModeName_Business  VisitModeName_Couples  VisitModeName_Family  \
0                         0.0                

In [92]:
# SEPARATING FEATURES (X) AND TARGET VARIABLE (Y) FOR MODEL TRAINING  
x = df_selected.drop(columns=["Rating"])
y = df_selected["Rating"]

In [93]:
x.head()

,VisitYear,VisitMonth,Attraction,CountryId,RegionId,VisitModeName_Business,VisitModeName_Couples,VisitModeName_Family,VisitModeName_Friends,VisitModeName_Solo,...,AttractionType_National Parks,AttractionType_Nature & Wildlife Areas,AttractionType_Neighborhoods,AttractionType_Points of Interest & Landmarks,AttractionType_Religious Sites,AttractionType_Spas,AttractionType_Speciality Museums,AttractionType_Volcanos,AttractionType_Water Parks,AttractionType_Waterfalls
0,2022,10,4.26703,163,21,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2022,10,4.26703,54,9,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2022,10,4.26703,51,8,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2022,9,4.26703,51,8,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2022,9,4.26703,115,16,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [94]:
scaler=StandardScaler()
x=scaler.fit_transform(x)

In [95]:
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42,test_size=0.2)

In [96]:
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "Decision Tree": DecisionTreeRegressor(random_state=42)
}

# --- Train, Predict, and Evaluate ---
for name, model in models.items():
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test,y_pred)

    print(f"\n {name}")
    print(f"Random Mean Squared Error: {rmse:.2f}")
    print(f"mae Score: {mae:.4f}")


 Linear Regression
Random Mean Squared Error: 0.92
mae Score: 0.7233

 Random Forest
Random Mean Squared Error: 0.97
mae Score: 0.7449

 Decision Tree
Random Mean Squared Error: 1.12
mae Score: 0.7993


In [ ]:
import xgboost as xgb

xgb_model = xgb.XGBRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

xgb_model.fit(x_train, y_train)

# --- Predictions and evaluation ---
y_pred = xgb_model.predict(x_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse=np.sqrt(mse)

print("XGBoost Regressor")
print("Mean Squared Error:", round(mse, 2))
print("R² Score:", round(r2, 4))
print('rmse',rmse)

print("After Random Training")


param_grid = {
    'n_estimators': [100, 200, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'subsample': [0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.7, 0.8, 0.9, 1.0]
}

xgb = xgb.XGBRegressor(random_state=42)
random_search = RandomizedSearchCV(xgb, param_distributions=param_grid, n_iter=20, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
random_search.fit(x_train, y_train)

best_xgb = random_search.best_estimator_

y_pred = best_xgb.predict(x_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"XGBoost Regressor (Tuned) → MAE: {mae:.3f}, RMSE: {rmse:.3f}")




: 

In [ ]:
import joblib
joblib.dump(best_xgb, 'xgboost_model.pkl')
joblib.dump(ohe,'ohe_encoder.pkl')
joblib.dump(te,'target_encoder.pkl')
joblib.dump(scaler,"scaler.pkl")

: 

NameError: name 'x_train' is not defined